### MQTT FUNCTIONS

In [10]:
# !pip install paho-mqtt python-dotenv 
# !pip install python-dotenv

In [11]:
import paho.mqtt.client as mqtt

import os
from dotenv import load_dotenv

load_dotenv()
client_name = os.environ.get('client')
token = os.environ.get('token')
server_name = os.environ.get('server')
port = os.environ.get('port')

In [12]:
server_name
token

'Makerjam2024'

In [13]:
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT broker")
    else:
        print(f"Connection failed with code {rc}")
        exit()


In [14]:
def mqtt_connects(data ):
        
    # Create an MQTT client instance
    client = mqtt.Client()
    # Set username and password for authentication
    username = client_name
    password = token
    client.username_pw_set(username, password)
    # Set the callback function
    client.on_connect = on_connect

    # Connect to the MQTT broker (replace with your broker's IP/hostname)
    broker_address = server_name
    client.connect(broker_address, int(port), 60)



    # Start the MQTT client loop in a non-blocking thread
    client.loop_start()

    try:
        previous_message = ""
        message = data


        # Publish a message to a specific topic

        if message != previous_message:
            # Publish a message to a specific topic
            topic = "/hand_d/get/data"
            client.publish(topic, message)
            
            print(f"Message published: {message}")
            previous_message = message
            # return True
        # topic = "test/topic"
        # client.publish(topic, message)
        else:
            print("Message unchanged. Not publishing.")

            # return False
        print("Message published. Disconnecting...")

    except Exception as e:
        print("An error occurred:", str(e))
        return False

    finally:
        # Disconnect the MQTT client
        client.loop_stop()
        client.disconnect()
        print("Disconnected from MQTT broker")



## Hand Detection Send Data

In [15]:
# !pip install opencv-python MediaPipe

In [16]:
import cv2
import mediapipe as mp
import numpy as np
from google.protobuf.json_format import MessageToDict

# MediaPipe hand detection module
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
# hand = mp_hands.Hands()



## Function Count And Send to MQTT

In [17]:
def process_frame(frame  ,old_count):
    # Flip the frame horizontally

    frame = cv2.flip(frame, 1)

    # Process the frame with MediaPipe
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Count the number of fingers
    finger_count = 0
    

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Calculate the finger count
            if   hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y:
                finger_count += 1
            if hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y:
                finger_count += 1
            if hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y:
                finger_count += 1
            if hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y:
                finger_count += 1
            if hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y:
                finger_count += 1

            # Draw hand landmarks on the frame
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Draw finger count on the frame
    cv2.putText(frame, f"Finger Count: {finger_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
     

    
    if (finger_count == old_count[0] ):
        already = True

    elif  (finger_count != old_count[0] ):
        old_count[0] = finger_count
        mqtt_connects(old_count[0])

    else:
        print( " - else - ")

    
    #Display the resulting frame
    cv2.namedWindow('Finger Count', cv2.WINDOW_NORMAL)  # Make the window resizable
    cv2.resizeWindow('Finger Count', 1024, 740) 
    cv2.imshow('Finger Count', frame)

## Functions ON Lamp


In [23]:
def process_frame_on_lamp(frame ,old_state  ):
    # Flip the frame horizontally
    # label_hand = ""
    frame = cv2.flip(frame, 1)
    # Process the frame with MediaPipe
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Count the number of fingers
    
    

    if results.multi_hand_landmarks:


        if len(results.multi_handedness) == 2:
                # Display 'Both Hands' on the image
            # send_mqtt_one_time(old_state,'Both Hands') 
            cv2.putText(frame, 'Both Hands', (250, 50),
            cv2.FONT_HERSHEY_COMPLEX,0.9, (0, 255, 0), 2)
        else:
            for i in results.multi_handedness:
                
                # Return whether it is Right or Left Hand
                label = MessageToDict(i)['classification'][0]['label']
  
                if label == 'Left':
                    
                    # Display 'Left Hand' on
                    # left side of window
                    cv2.putText(frame, label+' Hand',
                                (20, 50),
                                cv2.FONT_HERSHEY_COMPLEX, 
                                0.9, (0, 255, 0), 2)
                    
                    # send_mqtt_one_time(old_state,label)
  
                if label == 'Right':
                      
                    # Display 'Left Hand'
                    # on left side of window
                    cv2.putText(frame, label+' Hand', (450, 50),
                                cv2.FONT_HERSHEY_COMPLEX,
                                0.9, (0, 255, 0), 2)
 
                    
                    # send_mqtt_one_time(old_state,label)

        for handLms in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, handLms,mp_hands.HAND_CONNECTIONS )
    
    frame_resized = cv2.resize(frame, (1024, 740))  # Set width to 800px and height to 600px
    cv2.imshow('Hand ',frame_resized)



In [19]:
def check_status_change(previous_status, current_status):
    if previous_status != current_status:
        return True
    return False
def send_mqtt_one_time(old_state , now_state ):

    if check_status_change(old_state[0], now_state):
        if now_state == '':
            return 
        else:
            # print(old_state[0])

            old_state[0] = now_state 
            # print("Status has changed!")
            mqtt_connects(now_state)
            return 


## Function drawing line index 2 finger

In [20]:
def process_drawing_line_2_finger(frame ):
    # สร้าง canvas แบบโปร่งใส (เริ่มต้นเป็นสีดำ)
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    canvas = None
    prev_x, prev_y = None, None
    frame = cv2.flip(frame, 1)
    if canvas is None:
        canvas = np.zeros((h, w, 3), dtype=np.uint8)

    # แปลงภาพเป็น RGB สำหรับ MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # ดึงตำแหน่งของนิ้วชี้และนิ้วกลาง
            index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle_finger = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

            x_index, y_index = int(index_finger.x * w), int(index_finger.y * h)
            x_middle, y_middle = int(middle_finger.x * w), int(middle_finger.y * h)

            # คำนวณระยะห่างระหว่างนิ้วชี้และนิ้วกลาง
            distance = ((x_index - x_middle) ** 2 + (y_index - y_middle) ** 2) ** 0.5

            if distance < 40:
                # ลบเส้นเมื่อใช้นิ้วชี้และนิ้วกลางคู่กัน (วาดวงกลมสีดำ)
                cv2.circle(canvas, (x_index, y_index), 30, (0, 0, 0), -1)
            else:
                # วาดเส้นเมื่อใช้นิ้วชี้เพียงนิ้วเดียว
                if prev_x is not None and prev_y is not None:
                    cv2.line(canvas, (prev_x, prev_y), (x_index, y_index), (0, 0, 255), 5)
                prev_x, prev_y = x_index, y_index
    else:
        prev_x, prev_y = None, None

    # รวมภาพกล้องและ canvas แบบโปร่งใสเข้าด้วยกัน

    frame = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)
    cv2.namedWindow('Virtual Paint', cv2.WINDOW_NORMAL)  # Make the window resizable
    cv2.resizeWindow('Virtual Paint', 1024, 740) 
    cv2.imshow('Virtual Paint', frame)

## Function drawing index finger

In [21]:
def process_drawing_finger(frame, canvas, prev_x, prev_y, hands):
    h, w, _ = frame.shape
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            hand_label = handedness.classification[0].label

            if hand_label == 'Right':
                # วาดเส้นด้วยนิ้วชี้ของมือขวา
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                index_pip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP]

                x_index, y_index = int(index_tip.x * w), int(index_tip.y * h)

                # วาดเส้นเมื่อชูนิ้วชี้ขึ้น
                if index_tip.y < index_pip.y:
                    if prev_x is not None and prev_y is not None:
                        cv2.line(canvas, (prev_x, prev_y), (x_index, y_index), (0, 0, 255), 5)
                    prev_x, prev_y = x_index, y_index
                else:
                    prev_x, prev_y = None, None

            elif hand_label == 'Left':
                # ลบเส้นด้วยนิ้วชี้และนิ้วกลางของมือซ้ายคู่กัน
                index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                middle_finger = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

                x_index, y_index = int(index_finger.x * w), int(index_finger.y * h)
                x_middle, y_middle = int(middle_finger.x * w), int(middle_finger.y * h)

                distance = ((x_index - x_middle) ** 2 + (y_index - y_middle) ** 2) ** 0.5

                if distance < 40:
                    cv2.circle(canvas, (x_index, y_index), 30, (0, 0, 0), -1)

    else:
        prev_x, prev_y = None, None

    return canvas, prev_x, prev_y

## Run Functions

In [ ]:
# OpenCV video capture
cap = cv2.VideoCapture(0)
old_count = [0]
old_state = ["hand"]
# สร้าง Canvas โปร่งใส สำหรับการวาดเส้น
canvas = None
prev_x, prev_y = None, None
# Create a hands object
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.90,
    min_tracking_confidence=0.90) as hands:


    # Main program loop
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break
           
        # process_frame(frame  ,old_count ) # Finger Countช
        # process_frame_on_lamp(frame,old_state ) # Check Hand
        # process_drawing_line_2_finger(frame  ) # 2 index finger
        
        # วาดภาพ
        frame = cv2.flip(frame, 1)  # พลิกภาพให้เหมือนกระจก
        h, w, _ = frame.shape
        # ตรวจสอบและสร้าง canvas เมื่อยังไม่มีการสร้าง
        if canvas is None:
            canvas = np.zeros((h, w, 3), dtype=np.uint8)
        # เรียกใช้ฟังก์ชันวาดและรับค่า canvas และตำแหน่งนิ้วกลับมา
        canvas, prev_x, prev_y = process_drawing_finger(frame, canvas, prev_x, prev_y, hands)
        # รวมภาพกล้องและ canvas แบบโปร่งใสเข้าด้วยกัน
        frame = cv2.addWeighted(frame, 1, canvas, 1, 0)
        cv2.namedWindow('Drawing with Finger', cv2.WINDOW_NORMAL)  # Make the window resizable
        cv2.resizeWindow('Drawing with Finger', 1024, 740) 
        cv2.imshow('Drawing with Finger', frame)

       
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## TEST SCRIPS

In [ ]:
mqtt_connects("Test 22")

In [ ]:
def process_frame(frame ,already ,old_count):
    # Flip the frame horizontally

    frame = cv2.flip(frame, 1)

    # Process the frame with MediaPipe
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Count the number of fingers
    finger_count = 0
    

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Calculate the finger count
            if   hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y:
                finger_count += 1
            if hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y:
                finger_count += 1
            if hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y:
                finger_count += 1
            if hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y:
                finger_count += 1
            if hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y:
                finger_count += 1

            # Draw hand landmarks on the frame
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Draw finger count on the frame
    cv2.putText(frame, f"Finger Count: {finger_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
     
    # print( " - O - ")
    # print(old_count[0])
    # print( " - F - ")
    # print(old_count[1])

    
    if (finger_count == old_count[0] ):
        already = True
        # old_count[0] = finger_count
        # print( " - Old - ")
        # print(old_count[0])
    elif  (finger_count != old_count[0] ):
        old_count[0] = finger_count
    # if already:
        # print( " - xx - ")
        # print(old_count[0])

        mqtt_connects(old_count[0])
        # already = False
    else:
        print( " - else - ")

    
    #Display the resulting frame
    cv2.imshow('Finger Count', frame)
# OpenCV video capture
cap = cv2.VideoCapture(0)
data_already = False
old_count = [0]
# Create a hands object
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:


    # Main program loop
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        process_frame(frame ,data_already ,old_count )

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
class StatusStateMachine:
    def __init__(self):
        self.state = "On"

    def update_status(self, new_status):
        if new_status == "On" and self.state != "On":
            print("Status has changed to On!")
        elif new_status == "Off" and self.state != "Off":
            print("Status has changed to Off!")

In [ ]:
# Example usage
status_machine = StatusStateMachine()
status_machine.update_status("On")
status_machine.update_status("Off")

In [ ]:
def status_changed_handler(new_status):
    print(f"Status has changed to {new_status}!")

# Example usage
new_status = "Off"
status_changed_handler(new_status)

In [ ]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# ตั้งค่าหน้าจอการวาดภาพ
canvas = None
prev_x, prev_y = None, None

# เริ่มการจับภาพจากกล้อง
cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7) as hands:

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("ไม่สามารถอ่านภาพจากกล้องได้")
            break
        
        frame = cv2.flip(frame, 1)
        if canvas is None:
            canvas = frame.copy()

            

        # แปลงภาพเป็น RGB สำหรับ MediaPipe
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # ดึงตำแหน่งของนิ้วชี้และนิ้วกลาง
                index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                middle_finger = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

                h, w, _ = frame.shape
                x_index, y_index = int(index_finger.x * w), int(index_finger.y * h)
                x_middle, y_middle = int(middle_finger.x * w), int(middle_finger.y * h)

                # เช็คระยะห่างระหว่างนิ้วชี้และนิ้วกลาง
                distance = ((x_index - x_middle) ** 2 + (y_index - y_middle) ** 2) ** 0.5

                if distance < 40:
                    # ลบเส้นเมื่อใช้นิ้วชี้และนิ้วกลางคู่กัน
                    cv2.circle(canvas, (x_index, y_index), 30, (255, 255, 255), -1)
                else:
                    # วาดเส้นเมื่อใช้นิ้วชี้เพียงนิ้วเดียว
                    if prev_x is not None and prev_y is not None:
                        cv2.line(canvas, (prev_x, prev_y), (x_index, y_index), (0, 0, 255), 5)
                    prev_x, prev_y = x_index, y_index
        else:
            prev_x, prev_y = None, None

        # รวมภาพกล้องและภาพวาดเข้าด้วยกัน
        frame = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)
        cv2.namedWindow('Virtual Paint', cv2.WINDOW_NORMAL)  # Make the window resizable
        cv2.resizeWindow('Virtual Paint', 1024, 740) 
        cv2.imshow('Virtual Paint', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):  # กด 'ESC' เพื่อออก
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import mediapipe as mp

mp_hands = mp.solutions.hands

# เริ่มการจับภาพจากกล้อง
cap = cv2.VideoCapture(0)

# สร้างตัวตรวจจับมือ
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7) as hands:

    # สร้าง Canvas โปร่งใส สำหรับการวาดเส้น
    canvas = None
    prev_x, prev_y = None, None

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape

        # สร้าง canvas แบบโปร่งใส (เริ่มต้นเป็นสีดำ)
        if canvas is None:
            canvas = np.zeros((h, w, 3), dtype=np.uint8)

        # แปลงภาพเป็น RGB สำหรับ MediaPipe
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # ดึงตำแหน่งของนิ้วชี้และนิ้วกลาง
                index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                middle_finger = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

                x_index, y_index = int(index_finger.x * w), int(index_finger.y * h)
                x_middle, y_middle = int(middle_finger.x * w), int(middle_finger.y * h)

                # คำนวณระยะห่างระหว่างนิ้วชี้และนิ้วกลาง
                distance = ((x_index - x_middle) ** 2 + (y_index - y_middle) ** 2) ** 0.5

                if distance < 40:
                    # ลบเส้นเมื่อใช้นิ้วชี้และนิ้วกลางคู่กัน (วาดวงกลมสีดำ)
                    cv2.circle(canvas, (x_index, y_index), 30, (0, 0, 0), -1)
                else:
                    # วาดเส้นเมื่อใช้นิ้วชี้เพียงนิ้วเดียว
                    if prev_x is not None and prev_y is not None:
                        cv2.line(canvas, (prev_x, prev_y), (x_index, y_index), (0, 0, 255), 5)
                    prev_x, prev_y = x_index, y_index
        else:
            prev_x, prev_y = None, None

        # รวมภาพกล้องและ canvas แบบโปร่งใสเข้าด้วยกัน
        frame = cv2.addWeighted(frame, 1, canvas, 1, 0)

        cv2.imshow('Virtual Paint (No Frame Copy)', frame)

        if cv2.waitKey(1) & 0xFF == 27:  # กด 'ESC' เพื่อออก
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import mediapipe as mp

mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)

# เริ่มต้นค่าเริ่มต้น
canvas = None
prev_x, prev_y = None, None
# ฟังก์ชันสำหรับวาดด้วยนิ้วชี้และลบด้วยนิ้วชี้และนิ้วกลางคู่กัน
def process_drawing_finger(frame, canvas, prev_x, prev_y, hands):
    h, w, _ = frame.shape
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            hand_label = handedness.classification[0].label

            if hand_label == 'Right':
                # วาดเส้นด้วยนิ้วชี้ของมือขวา
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                index_pip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP]

                x_index, y_index = int(index_tip.x * w), int(index_tip.y * h)

                # วาดเส้นเมื่อชูนิ้วชี้ขึ้น
                if index_tip.y < index_pip.y:
                    if prev_x is not None and prev_y is not None:
                        cv2.line(canvas, (prev_x, prev_y), (x_index, y_index), (0, 0, 255), 5)
                    prev_x, prev_y = x_index, y_index
                else:
                    prev_x, prev_y = None, None

            elif hand_label == 'Left':
                # ลบเส้นด้วยนิ้วชี้และนิ้วกลางของมือซ้ายคู่กัน
                index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                middle_finger = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

                x_index, y_index = int(index_finger.x * w), int(index_finger.y * h)
                x_middle, y_middle = int(middle_finger.x * w), int(middle_finger.y * h)

                distance = ((x_index - x_middle) ** 2 + (y_index - y_middle) ** 2) ** 0.5

                if distance < 40:
                    cv2.circle(canvas, (x_index, y_index), 30, (0, 0, 0), -1)

    else:
        prev_x, prev_y = None, None

    return canvas, prev_x, prev_y
# สร้างตัวตรวจจับมือ
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.90,
    min_tracking_confidence=0.90) as hands:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)  # พลิกภาพให้เหมือนกระจก
        h, w, _ = frame.shape

        # ตรวจสอบและสร้าง canvas เมื่อยังไม่มีการสร้าง
        if canvas is None:
            canvas = np.zeros((h, w, 3), dtype=np.uint8)

        # เรียกใช้ฟังก์ชันวาดและรับค่า canvas และตำแหน่งนิ้วกลับมา
        canvas, prev_x, prev_y = process_drawing_finger(frame, canvas, prev_x, prev_y, hands)

        # รวมภาพกล้องและ canvas แบบโปร่งใสเข้าด้วยกัน
        frame = cv2.addWeighted(frame, 1, canvas, 1, 0)
        cv2.imshow('Drawing with Finger', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


